# Modelo

## Constantes

In [6]:
#NP = (110119 + 109164 + 111111) / 3 
NP = 110119 
A_TERR = 17.32 * NP / 3000

A_SOT = 8.66 ** 2                   # m2
Q_MAX = 8                           # m3/h
D_H_MAX = 3.5 # lo cambie           # m
D_H_MIN = 1                         # m
H_SOT = 3.50                        # m
V_SOT = H_SOT * A_SOT               # m3

C_SAT = 0.90                        # admin
C_0 = 0.60                          # admin

T_K = (1-NP/140000)                 # h


# EC 1
def f_der_v(q_ent, q_sal):
    return q_ent - q_sal

# EC 2
def f_q_ent(c, i, a_terr):
    return c * i * a_terr           # admin * m/h * m2 = m3/h

# EC 3
def f_q_sal(q_max, d_h_max, d_h_min, d_h):
    if d_h > d_h_max: return 0
    if d_h < d_h_min: return q_max
    return q_max * (((d_h_max - d_h)/(d_h_max - d_h_min)) ** 0.5)

# EC 4
def f_d_h(h_s, h):
    return h_s - h

# EC 5
def f_h(v, a_sot):
    return v / a_sot

# EC 6
def f_der_c(v, v_sot, t_k, c_sat, c):
    return (v/(v_sot * t_k)) * (c_sat - c)

## Métodos

In [7]:
def euler_explicito(f, h, t, u):            # m3
    return u + h * f(t, u)                  # m3 + h * m3/h

def euler_explicito_2(f, h, t, u, v):       # m3
    return u + h * f(t, u, v)               # m3 + h * m3/h

def euler_explicito_3(f, h, t, u, v, w):    # m3
    return u + h * f(t, u, v, w)            # m3 + h * m3/h

## A.1
Discretizar la ec. 1 con el método de Euler, tomando $C$ = 1, $Q$ = 0. Correr el modelo para la precipitación de 60 min y verificar que el volumen de agua almacenado en el sótano coincida con el volumen de lluvia.

In [8]:
INTENSITY = 85.0 / 1000  # m/h

DURATION = 1  # h
INFILTRATION = 1  # a-dimensional
Q_SAL = 0  # m3/h


def d_volume(t, u):
    q_ent = f_q_ent(INFILTRATION, INTENSITY, A_TERR)  # m3/h
    q_sal = Q_SAL  # m3/h

    return f_der_v(q_ent, q_sal)  # m3/h


d_time = 5 / 60  # h

volume = 0  # m3
time = 0  # h

while time < DURATION:
    volume = volume + d_time * d_volume(time, volume)
    time += d_time

print(f"Final volume: {round(volume, 2)} m3")

Final volume: 54.04 m3


## A.2

Discretizar las ec. 1 y ec. 6 con el método de Euler, considerando $C$ y $Q_{sal}$ variables. Correr
el modelo para todas las duraciónes / intensidades de precipitación, un lapso de tiempo
suficiente como para que el sótano se vacíe

In [11]:
storms_min_mm = [       # min, mm/h
    (5, 241.4),
    (10, 190.7),
    (15, 162.6),
    (30, 119.6),
    (60, 85.0),
    (180, 41.7),
    (360, 26.4),
    (720, 16.7),
    (1440, 10.9),
    (4320, 5.2),
]

storms = [(duration/60, intensity/1000) for duration, intensity in storms_min_mm]

def d_volume(time, volume, infiltration, intensity):
    terrain_area = A_TERR

    max_flow = Q_MAX
    max_height_variation = D_H_MAX
    min_height_variation = D_H_MIN
    
    sotane_height = H_SOT
    
    sotane_area = A_SOT
    height = f_h(volume, sotane_area)

    heigth_variation = f_d_h(sotane_height, height)

    entry_flow = f_q_ent(infiltration, intensity, terrain_area)
    exit_flow = f_q_sal(max_flow, max_height_variation, min_height_variation, heigth_variation)

    return f_der_v(entry_flow, exit_flow)

def d_infiltration(t, volume, infiltration):

    sotane_volume = V_SOT

    caracteristic_time = T_K
    saturation_infiltration = C_SAT

    return f_der_c(volume, sotane_volume, caracteristic_time, saturation_infiltration, infiltration)

for storm_duration, storm_intensity in storms:

    print(f"Starting storm with {storm_intensity*1000} mm/h for {storm_duration*60} min")

    time = 0
    volume = 0
    infiltration = C_0

    d_time = storm_duration / 100

    while time < storm_duration:
        volume = volume + d_time * d_volume(time, volume, infiltration, storm_intensity)
        infiltration = infiltration + d_time * d_infiltration(time, volume, infiltration)
        time += d_time

    print(f"Storm finish and sotane has {round(volume,2)} m3")

    while volume > 0:
        volume = volume + d_time * d_volume(time, volume, infiltration, 0)
        infiltration = infiltration + d_time * d_infiltration(time, volume, infiltration)
        time += d_time

    print(f"Empyting sotane after {round(time*60,2)} min")
    print("")


Starting storm with 241.4 mm/h for 5.0 min
Storm finish and sotane has 7.59 m3
Empyting sotane after 570.7 min

Starting storm with 190.7 mm/h for 10.0 min
Storm finish and sotane has 11.94 m3
Empyting sotane after 719.2 min

Starting storm with 162.6 mm/h for 15.0 min
Storm finish and sotane has 15.22 m3
Empyting sotane after 815.55 min

Starting storm with 119.6 mm/h for 30.0 min
Storm finish and sotane has 22.27 m3
Empyting sotane after 997.8 min

Starting storm with 85.0 mm/h for 60.0 min
Storm finish and sotane has 31.65 m3
Empyting sotane after 1213.2 min

Starting storm with 41.7 mm/h for 180.0 min
Storm finish and sotane has 47.37 m3
Empyting sotane after 1589.4 min

Starting storm with 26.4 mm/h for 360.0 min
Storm finish and sotane has 58.63 m3
Empyting sotane after 1926.0 min

Starting storm with 16.7 mm/h for 720.0 min
Storm finish and sotane has 65.14 m3
Empyting sotane after 2361.6 min

Starting storm with 10.9 mm/h for 1440.0 min
Storm finish and sotane has 62.0 m3
Empyt